In [1]:
import pandas as pd
import numpy
import dill
import sqlite3

1. Max number of items in one order: if a customer has more than one order, take  the one with more items.
2. Max revenue in one order: if a customer has more than one order, take the one  with largest revenue.  
3. Total revenue of a customer: including all orders 
4. Total number of orders   
5. Days since last order: number of days from the last order until 2017-10-17 
6. The longest interval between two consecutive orders (in unit of days). If a  customer has only one order, use the formula below

$Avg($longest_interval$) + $days_since_last_order


In [2]:
dill_file = open('model.dill', 'rb')
model = dill.load(dill_file)

In [3]:
model.predict(numpy.array([[3, 92.6, 109.3, 2, 12, 26],[2, 10.4, 43.5, 3, 26, 5]]))

array([ 244.9,   89.9])

In [4]:
def days_since_last_order(d):
    timedelta = pd.to_datetime('2017-10-17') - d.max()
    return timedelta.days

In [5]:
df = pd.read_csv('orders.csv', parse_dates=['created_at_date'])
df.columns

Index(['customer_id', 'order_id', 'order_item_id', 'num_items', 'revenue',
       'created_at_date'],
      dtype='object')

In [6]:
f_o = {
       'revenue': ['sum'], 
       'order_item_id': ['nunique'], 
       'created_at_date': ['max']
      }
f_c = {
       ('order_item_id','nunique'): ['max'], 
       ('revenue', 'sum'): ['max','sum'], 
       ('order_id', ''): ['nunique'], 
       ('created_at_date', 'max'): [days_since_last_order]
      }

In [7]:
res_o = df.groupby(['customer_id', 'order_id']).agg(f_o).reset_index()
res_c = res_o.groupby('customer_id').agg(f_c)

In [8]:
sorted_o = df.sort_values(['customer_id', 'created_at_date']).groupby(['customer_id', 'order_id'])['created_at_date'].max()
interval_c = sorted_o.groupby('customer_id').diff().groupby('customer_id').max().dt.days.rename(('created_at_date', 'diff', 'interval'))

In [9]:
res_c_i = res_c.merge(interval_c.to_frame(), left_index=True, right_index=True)

In [10]:
avg_longest_interval = interval_c.mean()

res_c_i.iloc[:, -1] = res_c_i.iloc[:,-1].fillna(avg_longest_interval + res_c_i.iloc[:, -2])

In [11]:
predictions = pd.Series.from_array(model.predict(res_c_i.as_matrix()))

res = res_c_i

res['predicted_clv'] = model.predict(res_c_i.as_matrix())

In [12]:
res['predicted_clv'].to_csv('result.csv', sep=',')

In [13]:
con = sqlite3.connect('data.sqlite')
res['predicted_clv'].to_sql('customers', con, if_exists='replace')
cur = con.cursor()
for row in cur.execute('SELECT * FROM customers LIMIT 10;'):
    print(row)
con.commit()
con.close()

('000011265b8a3727c4cc77b494134aca', 133.72534850355342)
('000079318ce4bdc1897a9c711e107b55', 128.90534850355343)
('0000e349e3d5eb96a2dc1939beebf9e4', 63.34534850355344)
('0001159afe8b9309aeb95fa17886d688', 94.26534850355344)
('000146a1e6decca6773a52afd2ed0255', 139.39999999999998)
('0001aa41356a707949e8a715947ff85c', 159.34534850355345)
('0001b3166c1bcb11f331e6676f7863a9', 113.28534850355344)
('0001cf68347245f00fe71fd9d677e9df', 66.66534850355343)
('00022da3f8257cbda32247d4e02c4b6d', 143.10534850355344)
('0002ec88abff6d01da3b9db674cd8868', 126.46534850355343)
